In [2]:
import torch
import torch.nn as nn
import memcnn

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [3]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [5]:
unet_model = UNet(n_channels=3, n_classes=10)

In [16]:
for layer in unet_model.children():
    print(layer)
    if isinstance(layer, nn.Conv2d):
        print(layer)

DoubleConv(
  (double_conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
  )
)
Down(
  (maxpool_conv): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace)
   

In [64]:
def dfs(module) : 
    if isinstance(module, nn.Module) : 
        if isinstance(module, torch.nn.Conv2d) : 
            print(module)
        else : 
            children_modules = list(module.children())
            #print(children_modules)
            #print("==================")
            for children in children_modules : 
                #print("child:", children)
                dfs(children)


In [65]:
dfs(unet_model)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), p

In [67]:
def dfs_module(module) : 
    if isinstance(module, nn.Module) : 
        if isinstance(module, torch.nn.Conv2d) : 
            print(module)
        else : 
            children_modules = list(module.children())
            #print(children_modules)
            #print("==================")
            for children in children_modules : 
                #print("child:", children)
                dfs(children)


{'_backend': <torch.nn.backends.thnn.THNNFunctionBackend at 0x7f6e824eb080>,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('inc',
               DoubleConv(
                 (double_conv): Sequential(
                   (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                   (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                   (2): ReLU(inplace)
                   (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                   (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                   (5): ReLU(inplace)
                 )
               )),
              ('down1',
               Down(
                 (maxpo

In [87]:
path_dict = {}
for key, value in unet_model._modules.items() :
    print(key, isinstance(value, torch.nn.Module))
    

inc True
down1 True
down2 True
down3 True
down4 True
up1 True
up2 True
up3 True
up4 True
outc True


In [77]:
model = 

True

In [88]:
for key, value in value._modules.items() : 
    print(key, value)

conv Conv2d(64, 10, kernel_size=(1, 1), stride=(1, 1))


In [89]:
value

Conv2d(64, 10, kernel_size=(1, 1), stride=(1, 1))

In [96]:
list(unet_model.modules())[0] = torch.nn.Linear(10,100)

In [103]:
list(unet_model.named_modules())[0]

('',
 UNet(
   (inc): DoubleConv(
     (double_conv): Sequential(
       (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU(inplace)
       (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (5): ReLU(inplace)
     )
   )
   (down1): Down(
     (maxpool_conv): Sequential(
       (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
       (1): DoubleConv(
         (double_conv): Sequential(
           (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
           (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU(inplace)
           (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
           (4): BatchNorm2d(128, eps=1e

In [102]:
unet_model._modules['inc']

DoubleConv(
  (double_conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
  )
)

In [18]:
in_channels=3
mid_channels=64
exmp_conv2d = torch.nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1)

In [23]:
input_example = torch.rand(10,in_channels,100,100)
output_example = exmp_conv2d(input_example)
print(input_example.shape, "-->conv2d-->", output_example.shape)

torch.Size([10, 3, 100, 100]) -->conv2d--> torch.Size([10, 64, 100, 100])


In [45]:
fm_input_channel = 1
gm_input_channel = 2

invertible_module = memcnn.AdditiveCoupling(
    Fm=torch.nn.Conv2d(fm_input_channel, mid_channels, kernel_size=3, padding=1),
    Gm=torch.nn.Conv2d(gm_input_channel, mid_channels, kernel_size=3, padding=1)
)

In [46]:

assert memcnn.is_invertible_module(invertible_module, test_input_shape=assert_input.shape)

RuntimeError: The size of tensor a (2) must match the size of tensor b (64) at non-singleton dimension 1

In [54]:
mid_channels=64
exmp_batchnorm = torch.nn.BatchNorm2d(mid_channels)
fm_input_channel = mid_channels // 2
gm_input_channel = mid_channels - fm_input_channel

invertible_module = memcnn.AdditiveCoupling(
    Fm=torch.nn.BatchNorm2d(fm_input_channel),
    Gm=torch.nn.BatchNorm2d(gm_input_channel),
)

In [64]:
input_example = torch.rand(10,mid_channels,100,100)
assert memcnn.is_invertible_module(invertible_module, test_input_shape=input_example.shape)

In [58]:
invertible_module_wrapper = memcnn.InvertibleModuleWrapper(fn=invertible_module, \
                                                    keep_input=False, keep_input_inverse=False)
#keek_input : forward의 input을 memory에 유지할지. False면 Backward에 역연산하여 구함
#keep_input_inverse : inverse연상의 input을 memory에 유지할지. False면 Backward에 역연산하여 구함

In [67]:
input_example = torch.rand(10,mid_channels,100,100)
copy_input = input_example.clone() #input_example이 forward될때 삭제되기 때문에, 검증용 복사본 만듬
Y_fw = invertible_module_wrapper.forward(input_example)
X_iv = invertible_module_wrapper.inverse(Y_fw)
print("reconstruction error :", torch.sum((copy_input - X_iv) ** 2.))

reconstruction error : tensor(1.4945e-08, grad_fn=<SumBackward0>)


In [69]:
mid_channels=64
exmp_batchnorm = torch.nn.BatchNorm2d(mid_channels)
fm_input_channel = mid_channels // 2
gm_input_channel = mid_channels - fm_input_channel

invertible_module = memcnn.AdditiveCoupling(
    Fm=torch.nn.BatchNorm2d(fm_input_channel),
    Gm=torch.nn.BatchNorm2d(gm_input_channel),
)

invertible_module_wrapper = memcnn.InvertibleModuleWrapper(fn=invertible_module, \
                                                    keep_input=False, keep_input_inverse=False)

input_example = torch.rand(10,mid_channels,100,100)
output_example = torch.rand(10,mid_channels,100,100)
mse = torch.nn.MSELoss()
criterion = torch.optim.Adam(invertible_module_wrapper.parameters(), lr=1e-2)

In [70]:
criterion.zero_grad()
Y_fw = invertible_module_wrapper.forward(input_example)
loss = mse(Y_fw, output_example)
loss.backward()
criterion.step()

In [106]:
import torchvision
import torchvision.transforms as transforms
cifar10 = torchvision.datasets.CIFAR10(root='../../20210306_gan/dataset', download=False,
                             transform=transforms.Compose([
                               transforms.Resize((32,32)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

In [107]:
cifar_loader =  torch.utils.data.DataLoader(cifar10, batch_size=1024, shuffle=True)

In [156]:
class Simple(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(Simple, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        
       
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=n_channels, out_channels=16, kernel_size=(3, 3), padding=1))
        self.pool = nn.MaxPool2d(kernel_size=(11,11), stride=(3,3))
        self.batch = nn.BatchNorm2d(num_features=16)
        self.activation = nn.ReLU()
        '''
        self.activation = memcnn.InvertibleModuleWrapper(fn= \
                             memcnn.AdditiveCoupling(
                                    Fm=torch.nn.ReLU(),
                                    Gm=torch.nn.ReLU(),
                             ), keep_input=False, keep_input_inverse=False)
        '''
        self.fcn = nn.Linear(16*8*8, n_classes)

    def forward(self, x):
        x1 = self.conv(x)
        x2 = self.pool(x1)
        x3 = self.batch(x2)
        x4 = self.activation(x3)
        x5 = self.fcn(x4.view(-1, 16*8*8))
        return x5

In [157]:
sp = Simple(n_channels=16, n_classes=10)

In [283]:
from collections import OrderedDict
import copy

def conv2d_to_invertible(block, inplace=True) : 
    replace_modules = copy.deepcopy(block._modules)     
    #print("before for loop", replace_modules)
    for i, (name, module) in enumerate(block.named_modules()) : 
        #print(i, name)
        if '.' not in name and isinstance(module, torch.nn.Conv2d) \
             and not isinstance(module, memcnn.InvertibleModuleWrapper):
            in_c = module.in_channels
            out_c = module.out_channels
            k = module.kernel_size
            s = module.stride
            p = module.padding
            d = module.dilation
            t = module.transposed
            op = module.output_padding
            g = module.groups
            if in_c == out_c : 
                #print(name, module, "\t\t-->")
                fm_input_size = in_c // 2
                gm_input_size = in_c - fm_input_size
                conv2d = memcnn.InvertibleModuleWrapper(fn= \
                             memcnn.AdditiveCoupling(
                                    Fm=torch.nn.Conv2d(fm_input_size, fm_input_size, k, s, p, d, g),
                                    Gm=torch.nn.Conv2d(gm_input_size, gm_input_size, k, s, p, d, g),
                             ), keep_input=False, keep_input_inverse=False)
                replace_modules[name] = conv2d
                #print(conv2d)
            else : 
               #print("input dim and output dim is not matched")
                pass
    #print("after for loop", replace_modules)       
    block._modules = replace_modules

In [316]:
def dfs_conv2d_to_invertible(top_module, inplace=True) : 
    conv2d_to_invertible(top_module, inplace=True)
    for name, module in top_module._modules.items() : 
        if isinstance(module, memcnn.InvertibleModuleWrapper) : 
            continue
        print(name, len(module._modules))
        if len(module._modules) > 0 : 
            dfs_conv2d_to_invertible(module)

In [327]:
top_module = torch.nn.Sequential(torch.nn.Conv2d(6,6,3), 
                             torch.nn.Sequential(
                                 torch.nn.Conv2d(8,8,3),
                                 torch.nn.Conv2d(10,10,3)  ),
                             torch.nn.Sequential(
                                 torch.nn.Conv2d(12,12,4),
                                 torch.nn.Sequential(
                                     torch.nn.Conv2d(14,14,4)))
                            )


In [331]:
unet_model

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=

In [332]:
dfs_conv2d_to_invertible(unet_model)

inc 1
double_conv 6
0 0
1 0
2 0
4 0
5 0
down1 1
maxpool_conv 2
0 0
1 1
double_conv 6
0 0
1 0
2 0
4 0
5 0
down2 1
maxpool_conv 2
0 0
1 1
double_conv 6
0 0
1 0
2 0
4 0
5 0
down3 1
maxpool_conv 2
0 0
1 1
double_conv 6
0 0
1 0
2 0
4 0
5 0
down4 1
maxpool_conv 2
0 0
1 1
double_conv 6
1 0
2 0
4 0
5 0
up1 2
up 0
conv 1
double_conv 6
0 0
1 0
2 0
3 0
4 0
5 0
up2 2
up 0
conv 1
double_conv 6
0 0
1 0
2 0
3 0
4 0
5 0
up3 2
up 0
conv 1
double_conv 6
0 0
1 0
2 0
3 0
4 0
5 0
up4 2
up 0
conv 1
double_conv 6
0 0
1 0
2 0
4 0
5 0
outc 1
conv 0


In [333]:
unet_model

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): InvertibleModuleWrapper(
        (_fn): AdditiveCoupling(
          (Gm): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (Fm): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [293]:
_ = conv2d_to_invertible(module, inplace=True)

In [294]:
module.__dict__

{'_backend': <torch.nn.backends.thnn.THNNFunctionBackend at 0x7f6e824eb080>,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('0',
               InvertibleModuleWrapper(
                 (_fn): AdditiveCoupling(
                   (Gm): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1))
                   (Fm): Conv2d(5, 5, kernel_size=(3, 3), stride=(1, 1))
                 )
               )),
              ('1',
               Sequential(
                 (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
                 (1): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1))
               )),
              ('2',
               Sequential(
                 (0): Conv2d(12, 12, kernel_size=(4, 4), stride=(1, 1))
                 (1): Sequential(
             

In [ ]:
def dfs_conv2d_to_invertible(module) : 
    if hasattr(module, '_modules') :
        if module._modules

In [233]:
invertible_unet = conv2d_to_invertible(unet_model)


inc
inc.double_conv
inc.double_conv.0
inc.double_conv.1
inc.double_conv.2
inc.double_conv.3
inc.double_conv.4
inc.double_conv.5
down1
down1.maxpool_conv
down1.maxpool_conv.0
down1.maxpool_conv.1
down1.maxpool_conv.1.double_conv
down1.maxpool_conv.1.double_conv.0
down1.maxpool_conv.1.double_conv.1
down1.maxpool_conv.1.double_conv.2
down1.maxpool_conv.1.double_conv.3
down1.maxpool_conv.1.double_conv.4
down1.maxpool_conv.1.double_conv.5
down2
down2.maxpool_conv
down2.maxpool_conv.0
down2.maxpool_conv.1
down2.maxpool_conv.1.double_conv
down2.maxpool_conv.1.double_conv.0
down2.maxpool_conv.1.double_conv.1
down2.maxpool_conv.1.double_conv.2
down2.maxpool_conv.1.double_conv.3
down2.maxpool_conv.1.double_conv.4
down2.maxpool_conv.1.double_conv.5
down3
down3.maxpool_conv
down3.maxpool_conv.0
down3.maxpool_conv.1
down3.maxpool_conv.1.double_conv
down3.maxpool_conv.1.double_conv.0
down3.maxpool_conv.1.double_conv.1
down3.maxpool_conv.1.double_conv.2
down3.maxpool_conv.1.double_conv.3
down3.maxpo

In [226]:
invertible_unet

UNet()

In [231]:
list(unet_model._modules.named_modules())

AttributeError: 'collections.OrderedDict' object has no attribute 'named_modules'

In [211]:
conv2d_to_invertible

In [240]:
module = torch.nn.Conv2d(11,11,3)

In [241]:
inv = conv2d_to_invertible(module, inplace=True)

NameError: name 'breakpoint' is not defined

In [238]:
inv

Conv2d(
  11, 11, kernel_size=(3, 3), stride=(1, 1)
  (): InvertibleModuleWrapper(
    (_fn): AdditiveCoupling(
      (Gm): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1))
      (Fm): Conv2d(5, 5, kernel_size=(3, 3), stride=(1, 1))
    )
  )
)

In [129]:
module.kernel_size
module.stride
module.padding
module.dilation
module.transposed
module.output_padding
module.groups

1

In [108]:
data, label = next(iter(cifar_loader))

In [126]:
model = Simple(n_channels=3, n_classes=10).to('cuda')
print(model)

Simple(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(11, 11), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (batch): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): InvertibleModuleWrapper(
    (_fn): AdditiveCoupling(
      (Gm): ReLU()
      (Fm): ReLU()
    )
  )
  (fcn): Linear(in_features=1024, out_features=10, bias=True)
)


In [127]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [131]:
epoch = 0
while epoch<1e+4 : 
    for data, label in cifar_loader : 
        predict = model(data.to('cuda'))
        loss = criterion(predict, label.to('cuda'))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    epoch += 1
    print(epoch, loss.item())

1 1.4968420267105103
2 1.3536299467086792
3 1.3239946365356445
4 1.2239649295806885
5 1.2539869546890259
6 1.1471829414367676
7 1.1970001459121704
8 1.1888761520385742
9 1.2050492763519287
10 1.099938988685608
11 1.1242170333862305
12 1.2335519790649414
13 1.2399179935455322
14 1.2075155973434448
15 1.1155601739883423
16 1.0674936771392822
17 1.0859053134918213
18 1.094795823097229
19 1.0883667469024658
20 1.0919687747955322
21 1.1286555528640747


KeyboardInterrupt: 

In [ ]:
class Simple_revrelu(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(Simple, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.conv = nn.Conv2d(in_channels=n_channels, out_channels=16, kernel_size=(3, 3), padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(11,11), stride=(3,3))
        self.batch = nn.BatchNorm2d(num_features=16)
        self.activation = nn.ReLU(inplace=True)
        self.fcn = nn.Linear(16*8*8, n_classes)

    def forward(self, x):
        x1 = self.conv(x)
        x2 = self.pool(x1)
        x3 = self.batch(x2)
        x4 = self.activation(x3)
        x5 = self.fcn(x4.view(-1, 16*8*8))
        #print(x.shape, x1.shape, x2.shape, x3.shape, x4.shape, x5.shape)
        return x5